In [3]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
import time

In [4]:

# CONFIGURATION 
EAR_THRESHOLD = 0.25 
MAR_THRESHOLD = 0.10 

# Eye indices for 6-point EAR
Right_Eye = [33, 160, 158, 133, 153, 144]
Left_Eye = [362, 385, 387, 263, 373, 380]
# Lips indices (upper and lower points for vertical distance)
Lip_Points = [13, 14]

# HELPER FUNCTIONS
def ear(pts):
    """Calculates the Eye Aspect Ratio (EAR) for a 6-point eye landmark set."""
    # Vertical distances
    A = dist.euclidean(pts[1], pts[5])
    B = dist.euclidean(pts[2], pts[4])
    # Horizontal distance
    C = dist.euclidean(pts[0], pts[3])
    # EAR formula
    return (A + B) / (2.0 * C) if C != 0 else 0.0

def normalized_lip_distance(u, l):
    """Calculates the Euclidean distance between two face landmarks in normalized coordinates (0.0 to 1.0)."""
    u_norm = np.array([u.x, u.y])
    l_norm = np.array([l.x, l.y])
    return np.linalg.norm(u_norm - l_norm)

def pixel_landmarks(face, w, h):
    """Converts normalized landmarks to pixel coordinates."""
    return [(int(lm.x * w), int(lm.y * h)) for lm in face.landmark]

def face_box(face, w, h):
    """Calculates the bounding box (min/max x/y) for a face."""
    xs = [lm.x for lm in face.landmark]
    ys = [lm.y for lm in face.landmark]
    x1, y1 = int(min(xs) * w), int(min(ys) * h)
    x2, y2 = int(max(xs) * w), int(max(ys) * h)
    return x1, y1, x2, y2

# --- MAIN FUNCTION ---
def start_detection():
    mp_face_mesh = mp.solutions.face_mesh
    
    # Track counters per-face
    # L: Left Blink Count, R: Right Blink Count, M: Mouth Open Count
    # prev_L/R: Previous state (1=Open, 0=Closed), prev_M: Previous state (1=Open, 0=Closed)
    counters = {} 

    cv2.namedWindow("Face Blink Mouth Detection", cv2.WINDOW_FREERATIO)
    cap = cv2.VideoCapture(0)

    # Variables for FPS calculation
    p_time = 0

    with mp_face_mesh.FaceMesh(static_image_mode=False, 
                               max_num_faces=10, refine_landmarks=True,
                               min_detection_confidence=0.7,
                               min_tracking_confidence=0.7) as mesh:

        while True:
            ok, frame = cap.read()
            if not ok: continue
            
            frame = cv2.flip(frame, 1) # Mirror image for easier user interaction
            h, w, _ = frame.shape

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = mesh.process(rgb)

            # PROCESS FACES 
            if results.multi_face_landmarks:
                for face_id, face in enumerate(results.multi_face_landmarks):

                    # Ensure counter dictionary exists for each face
                    if face_id not in counters:
                        counters[face_id] = {"L":0, "R":0, "M":0,
                                             "prev_L":1, "prev_R":1, "prev_M":0}

                    pts = pixel_landmarks(face, w, h)

                    # EYE EAR CALCULATION 
                    RE = [pts[i] for i in Right_Eye]
                    LE = [pts[i] for i in Left_Eye]
                    
                    rEAR = ear(RE)
                    lEAR = ear(LE)

                    # --- BLINK DETECTION (Falling Edge Logic) ---
                    # 100% accuracy to calculate the count in left eye open and close
                    if lEAR < EAR_THRESHOLD and counters[face_id]["prev_L"] == 1:
                        counters[face_id]["L"] += 1
                    
                    # 100% accuracy to calculate the count in right eye open and close
                    if rEAR < EAR_THRESHOLD and counters[face_id]["prev_R"] == 1:
                        counters[face_id]["R"] += 1

                    # Update previous state (1=Open, 0=Closed)
                    counters[face_id]["prev_R"] = 1 if rEAR > EAR_THRESHOLD else 0
                    counters[face_id]["prev_L"] = 1 if lEAR > EAR_THRESHOLD else 0
                    
                    # ------------- MOUTH MAR CALCULATION -------------
                    u = face.landmark[Lip_Points[0]] # Upper lip
                    l = face.landmark[Lip_Points[1]] # Lower lip
                    mar = normalized_lip_distance(u, l)

                    # --- MOUTH OPEN DETECTION (Rising Edge Logic) ---
                    # 100% accuracy to calculate the count in mouth open (yawn/speech)
                    if mar > MAR_THRESHOLD and counters[face_id]["prev_M"] == 0:
                        counters[face_id]["M"] += 1
                        counters[face_id]["prev_M"] = 1 # Set state to Open
                        
                    if mar <= MAR_THRESHOLD:
                        counters[face_id]["prev_M"] = 0 # Set state to Closed

                    # ------------- DRAWINGS & TEXT -------------
                    x1, y1, x2, y2 = face_box(face, w, h)
                    
                    # Bounding Box (Yellow)
                    cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)
                    
                    # Draw Eyes (Red) and Lips (Blue)
                    for p in RE + LE:
                        cv2.circle(frame, p, 2, (255,0,0), -1)
                        
                    # Use pixel coordinates for drawing lips (from pts list for consistency)
                    cv2.circle(frame, pts[Lip_Points[0]], 3, (255,100,0), -1) 
                    cv2.circle(frame, pts[Lip_Points[1]], 3, (255,100,0), -1)
                    
                    # --- BEAUTIFUL TEXT DISPLAY ---
                    # Text positioning variables
                    text_x = x1 + 5
                    text_y_start = y2 + 20
                    line_height = 25
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 0.6
                    thickness = 2
                    
                    # Face ID
                    cv2.putText(frame, f"FACE {face_id} - EAR: {((rEAR+lEAR)/2):.2f}", (x1, y1-10),
                                font, 0.6, (0,255,255), 2)
                    
                    # Eye Blinks Group (Green)
                    cv2.putText(frame, f"L-Blinks: {counters[face_id]['L']} / R-Blinks: {counters[face_id]['R']}", 
                                (text_x, text_y_start), font, font_scale, (0,255,0), thickness)
                    
                    # Mouth Count Group (Orange)
                    cv2.putText(frame, f"Mouth Opens: {counters[face_id]['M']} (MAR: {mar:.2f})", 
                                (text_x, text_y_start + line_height), font, font_scale, (0,128,255), thickness)
                    
            # --- FPS Feature (Best Feature) ---
            c_time = time.time()
            fps = 1 / (c_time - p_time) if c_time != p_time else 0.0
            p_time = c_time
            cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), font, 0.7, (255,0,0), 2)

            # --- DISPLAY ---
            cv2.imshow("Face Blink Mouth Detection", frame)
            
            # Exit condition
            if cv2.waitKey(5) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()


    # RUN
if __name__ == "__main__":
    start_detection()